### Sentiment Engine training

In [1]:
!pip install transformers
from transformers import RobertaForSequenceClassification, RobertaTokenizer, BertForSequenceClassification, BertTokenizer, AutoModelForSequenceClassification, AutoTokenizer, AdamW

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 36.5 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 42.7 MB/s 
     |████████████████████████████████| 895 kB 27.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/MyDrive/Projects/PLP_PracticeModule/

/content/drive/MyDrive/Projects/PLP_PracticeModule


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
%matplotlib inline

In [5]:
df = pd.read_csv('final_dataset_for_training.csv',usecols=['body','entities','Stock'])
df

,body,entities,Stock
0,$TSLA stock starting to rocket 🚀,Bullish,TSLA
1,$TSLA Money got banana it grow big \nBut took ...,Bullish,TSLA
2,"$AAPL bear logic last qtr, but the iPhone numb...",Bullish,AAPL
3,$AAPL 129 at the morning bell. Sleep well my B...,Bullish,AAPL
4,$AMZN $TSLA $AAPL $GOOGL now well get some che...,Bullish,TSLA
...,...,...,...
3199995,$TSLA short the rip until we hit $900,Bearish,TSLA
3199996,$AAPL,Bearish,AAPL
3199997,$AMZN think its going to fill 3670 first...? T...,Bearish,AMZN
3199998,@LuckyLloydChristmas @Bullytrading Im such a f...,Bearish,TSLA


In [6]:
df['entities'] = df.entities.replace('Bullish',1).replace('Bearish',0)

In [7]:
!pip install emoji

     |████████████████████████████████| 175 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=e04dbe1ec3fcaf247ee9a493535fdca940838761347e1f7e3e3e4ba64c55feb0
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [8]:
import emoji
import re

def process_text(message):
  # lowercase
  # message = message.lower() # RoBERTa tokenizer is uncased
  # remove URLs
  message = re.sub(r'https?://\S+', "", message)
  # remove symbol names
  message = re.sub(r"\$\S+", "cashtag", message)
  message = re.sub(r"\#\S+", "hashtag", message)
  # remove usernames
  message = re.sub(r"\@\S+", "entity", message)
  # demojize
  message = emoji.demojize(message, delimiters=("", " "))


  return message.strip()
    

In [9]:
df['body'] = df.body.apply(process_text)
df.head()

,body,entities,Stock
0,cashtag stock starting to rocket rocket,1,TSLA
1,cashtag Money got banana it grow big \nBut too...,1,TSLA
2,"cashtag bear logic last qtr, but the iPhone nu...",1,AAPL
3,cashtag 129 at the morning bell. Sleep well my...,1,AAPL
4,cashtag cashtag cashtag cashtag now well get s...,1,TSLA


In [10]:
labels = df.entities.values
text = df.body.values

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
len(tokenizer)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize
def helper(text):
  t = word_tokenize(text)
  return len(t)
length = df.body.apply(helper)

In [ ]:
print(len(length[length>32]))
print(len(length[length>64]))
print(len(length[length>128]))

In [ ]:
input_ids = []
attention_mask = []
for i in text:
    encoded_data = tokenizer.encode_plus(
    i,
    add_special_tokens=True,
    truncation=True,
    max_length=64,
    padding='max_length',
    return_attention_mask= True,
    return_tensors='pt')
    input_ids.append(encoded_data['input_ids'])
    attention_mask.append(encoded_data['attention_mask'])
input_ids = torch.cat(input_ids,dim=0)
attention_mask = torch.cat(attention_mask,dim=0)
labels = torch.tensor(labels)

In [ ]:
from torch.utils.data import DataLoader,SequentialSampler,RandomSampler,TensorDataset,random_split

dataset = TensorDataset(input_ids,attention_mask,labels)
train_size = int(0.9*len(dataset))
val_size = len(dataset) - train_size

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

print('Training Size - ',train_size)
print('Validation Size - ',val_size)

In [ ]:
train_dl = DataLoader(train_dataset,sampler = RandomSampler(train_dataset),
                     batch_size = 32)
val_dl = DataLoader(val_dataset,sampler = SequentialSampler(val_dataset),
                     batch_size = 32)
len(train_dl),len(val_dl)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 2, output_attentions = False, output_hidden_states = False)

In [ ]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

In [ ]:
optimizer = AdamW(model.parameters(),lr = 2e-5,eps=1e-8,no_deprecation_warning=True)

In [ ]:
from transformers import get_linear_schedule_with_warmup
epochs = 1
total_steps = len(train_dl)*epochs
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)

In [ ]:
def accuracy(preds,labels):
    pred_flat = np.argmax(preds,axis=1).flatten()
    label_flat = labels.flatten()
    return np.sum(pred_flat==label_flat)/len(label_flat)

In [ ]:
def evaluate(dataloader_test):
    model.eval()
    loss_val_total = 0
    predictions,true_vals = [],[]
    for batch in dataloader_test:
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids':batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        with torch.no_grad():
            outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    loss_val_avg = loss_val_total / len(dataloader_test)
    predictions = np.concatenate(predictions,axis=0)
    true_vals = np.concatenate(true_vals,axis=0)
    return loss_val_avg,predictions,true_vals

In [ ]:
from tqdm.notebook import tqdm
torch.cuda.empty_cache()
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(train_dl, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dl)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(val_dl)
    val_acc = accuracy(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Accuracy: {val_acc}')

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/90000 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.34953850715657075
Validation loss: 0.2956085407484323
Accuracy: 0.86789375


In [ ]:
output_dir = 'sentimentEngine/'
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('sentimentEngine/tokenizer_config.json',
 'sentimentEngine/special_tokens_map.json',
 'sentimentEngine/vocab.json',
 'sentimentEngine/merges.txt',
 'sentimentEngine/added_tokens.json')

### RobertaForSequenceClassification

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

### Use sentiment engine

In [ ]:
!pip install transformers
!pip install emoji

In [82]:
import transformers
import torch
import math
import pandas as pd
import numpy as np
from transformers import RobertaForSequenceClassification, RobertaTokenizer, BertForSequenceClassification, BertTokenizer, AutoModelForSequenceClassification, AutoTokenizer, AdamW
import random
import time

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [22]:
cd drive/MyDrive/Projects/PLP_PracticeModule/

/content/drive/MyDrive/Projects/PLP_PracticeModule


In [33]:
filepath = 'sentimentEngine/'
tokenizer_loaded = RobertaTokenizer.from_pretrained(filepath)
model_loaded = RobertaForSequenceClassification.from_pretrained(filepath)

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def Sentiment(sent,model=model_loaded,tokenizer=tokenizer_loaded):
  encoded_dict = tokenizer.encode_plus(
                      sent, 
                      add_special_tokens = True,
                      truncation=True,
                      max_length = 64,
                      padding='max_length',
                      return_attention_mask = True,
                      return_tensors = 'pt')
      
  input_id = torch.LongTensor(encoded_dict['input_ids']).to(device)
  attention_mask = torch.LongTensor(encoded_dict['attention_mask']).to(device)
  model = model.to(device)

  with torch.no_grad():
      outputs = model(input_id, token_type_ids=None, attention_mask=attention_mask)

  logits = outputs[0]
  index = logits.argmax()
  return index,logits

import emoji
import re

def process_text(message):
  # lowercase
  # message = message.lower() # RoBERTa tokenizer is case-sensitive, finBERT and BERT we used uncased tokenizer
  # remove URLs
  message = re.sub(r'https?://\S+', "", message)
  # remove symbol names
  message = re.sub(r"\$\S+", "cashtag", message)
  message = re.sub(r"\#\S+", "hashtag", message)
  # remove usernames
  message = re.sub(r"\@\S+", "entity", message)
  # demojize
  message = emoji.demojize(message, delimiters=("", " "))

  return message.strip()

def checkSenti(sent,return_logits=False):
  labels = ['Bearish','Bullish']
  sent_processed = process_text(sent)
  index,logits = Sentiment(sent_processed)
  if return_logits:
    logit0 = math.exp(logits[0][0])
    logit1 = math.exp(logits[0][1])
    logits = [logit0/(logit0+logit1),logit1/(logit0+logit1)]
    return labels[index],logits
  return labels[index]


In [107]:
from torch.utils.data import DataLoader,SequentialSampler,TensorDataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batch_checkSenti(texts,model=model_loaded,tokenizer=tokenizer_loaded,return_logits=False):
  start = time.time()
  labels = ['Bearish','Bullish']
  input_ids = []
  attention_masks = []
  for text in texts:
      i = process_text(text)
      encoded_data = tokenizer.encode_plus(
      i,
      add_special_tokens=True,
      truncation=True,
      max_length=64,
      padding='max_length',
      return_attention_mask= True,
      return_tensors='pt')
      input_ids.append(encoded_data['input_ids'])
      attention_masks.append(encoded_data['attention_mask'])
  input_ids = torch.cat(input_ids,dim=0)
  attention_masks = torch.cat(attention_masks,dim=0)
  model.to(device)
  print('It takes {}s to tokenize'.format(time.time()-start))
  checkpointtime = time.time()

  testset = TensorDataset(input_ids,attention_masks)
  test_dl = DataLoader(testset,sampler = SequentialSampler(testset),batch_size = 128)

  predictions = []
  for batch in test_dl:
      batch = tuple(b.to(device) for b in batch)
      inputs = {
          'input_ids':batch[0],
          'attention_mask': batch[1],
      }

      with torch.no_grad():
          outputs = model(**inputs)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      predictions.append(logits)

  predictions = np.concatenate(predictions,axis=0)
  index = predictions.argmax(axis=1)
  print('It takes {}s to do predictions'.format(time.time()-checkpointtime))

  # if return_logits:
  #   return index,predictions
  return (index,predictions) if return_logits else index

In [109]:
samples_num = 1280
testing = batch_checkSenti(df.body.iloc[:samples_num])
true_labels = df.entities.iloc[:samples_num].replace('Bullish',1).replace('Bearish',0)
# len(['same' for i in range(samples_num) if testing[i]==true_labels[i]])/samples_num
testing

It takes 0.6091609001159668s to tokenize
It takes 2.363013505935669s to do predictions


array([1, 1, 1, ..., 1, 1, 0])

In [106]:
samples = ['just buy','just sell it','entity rocket to the sky!','go down','even though it is going up, I still think it will go down in the near future']
for sample in samples:
  print(checkSenti(sample))

Bullish
Bearish
Bullish
Bearish
Bearish


#### test 10000 samples and check confusion matrix

In [36]:
df = pd.read_csv('testset20000.csv')

In [ ]:
df

In [95]:
# sentiments = []
# true_labels = np.array(df.entities[1595000:1605000])
# for i,text in enumerate(df.body[1595000:1605000]):
#   sentiments.append(checkSenti(text))
#   if i %100 == 0:
#     print(i)

sentiments = []
true_labels = np.array(df.entities.iloc[:1280])
for i,text in enumerate(df.body.iloc[:1280]):
  sentiments.append(checkSenti(text))
  if i%1000 == 0:
    print(i)

0
1000


In [ ]:
len([item for item in true_labels if item == 'Bullish'])

10000

In [ ]:
true_pos,true_neg,false_pos,false_neg = 0,0,0,0
for i in range(20000):
  if true_labels[i] == 'Bullish':
    if sentiments[i] == 'Bullish':
      true_pos += 1
    else:
      false_neg += 1
  else:
    if sentiments[i] == 'Bearish':
      true_neg += 1
    else:
      false_pos += 1

In [ ]:
print('    True labels')
print('    pos   neg')
print('pos  {}    {}'.format(true_pos,false_pos))
print('neg  {}    {}'.format(false_neg,true_neg))

    True labels
    pos   neg
pos  8339    1344
neg  1661    8656


In [ ]:
total = true_pos+true_neg+false_pos+false_neg
accuracy = (true_pos+true_neg)/total
precision = true_pos/(true_pos+false_pos)
recall = true_pos/(true_pos+false_neg)
f1 = 2*precision*recall/(precision+recall)
print('accuracy = ',accuracy)
print('precision = ',precision)
print('recall = ',recall)
print('f1 score = ',f1)

accuracy =  0.84975
precision =  0.8612000413095116
recall =  0.8339
f1 score =  0.8473301834070011


#### Models performance we trained to test on stocktwits data

**RoBERTa(fine tuned on stocktwits) on unseen testset**

**accuracy =  0.84975**

**precision =  0.8612000413095116**

**recall =  0.8339**

**f1 score =  0.8473301834070011**

RoBERTa(fine tuned on stocktwits)

accuracy =  0.8866

precision =  0.9020178905762429

recall =  0.867373474694939

f1 score =  0.8843565164185192

RoBERTa(fine tuned on 1.6m tweets)

accuracy =  0.6249

precision =  0.5970775687859474

recall =  0.7682

f1 score =  0.6719146330796816

BERT(fine tuned on 1.6m tweets)

accuracy =  0.6165

precision =  0.5983454330575722

recall =  0.7088

f1 score =  0.6489059782111141

FinBERT(fine tuned on 1.6m tweets)

accuracy =  0.6124

precision =  0.5935419440745673

recall =  0.7132

f1 score =  0.6478924418604652